In [1]:
from collections import Counter, defaultdict
from gensim.models import Word2Vec
from IPython import display
from nltk import word_tokenize
from nltk.translate.bleu_score import sentence_bleu
from PIL import Image
from torch import nn
from torch.autograd import Variable
from torchvision import models, transforms

import json
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import random
import torch
import torch.nn.functional as F

C:\Users\belin\Anaconda3\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


# Data Acquisition

For this assignment, you will reuse the dataset you downloaded in assignment 2. This dataset contains a very large set of images, approximately 80K training images and 100 validation images, with multiple tags for each image. However that data *lacks captions* for the images, which is **vital** for this assignment. To obtain the captions for this assignment, download a few data files as shown below and add them to your `data/annotations` folder from assignment 2.

`wget https://s3-us-west-2.amazonaws.com/cpsc532l-data/a4_data.zip`

Following the data downloading and unzipping, the code below loads in the data into memory accordingly.

In [2]:
# Define a global transformer to appropriately scale images and subsequently convert them to a Tensor.
img_size = 224
loader = transforms.Compose([
  transforms.Resize(img_size),
  transforms.CenterCrop(img_size),
  transforms.ToTensor(),
]) 
def load_image(filename, volatile=False):
    """
    Simple function to load and preprocess the image.

    1. Open the image.
    2. Scale/crop it and convert it to a float tensor.
    3. Convert it to a variable (all inputs to PyTorch models must be variables).
    4. Add another dimension to the start of the Tensor (b/c VGG expects a batch).
    5. Move the variable onto the GPU.
    """
    image = Image.open(filename).convert('RGB')
    image_tensor = loader(image).float()
    image_var = Variable(image_tensor, volatile=volatile).unsqueeze(0)
    return image_var.cuda()

load_image('data/train2014/COCO_train2014_000000000009.jpg')

Variable containing:
( 0 , 0 ,.,.) = 
  0.0039  0.0078  0.0039  ...   0.0471  0.0471  0.0314
  0.0039  0.0039  0.0039  ...   0.0353  0.0353  0.0392
  0.0039  0.0039  0.0039  ...   0.0392  0.0392  0.0510
           ...             ⋱             ...          
  0.7137  0.7294  0.7137  ...   0.1686  0.1843  0.1686
  0.7059  0.6902  0.6863  ...   0.1765  0.1804  0.2039
  0.6784  0.6667  0.6706  ...   0.1922  0.2157  0.2275

( 0 , 1 ,.,.) = 
  0.1490  0.1490  0.1412  ...   0.0039  0.0039  0.0039
  0.1451  0.1412  0.1373  ...   0.0039  0.0039  0.0039
  0.1412  0.1373  0.1373  ...   0.0039  0.0039  0.0039
           ...             ⋱             ...          
  0.4392  0.4667  0.4549  ...   0.2588  0.2745  0.2863
  0.4353  0.4235  0.4196  ...   0.2745  0.2980  0.3137
  0.4118  0.4000  0.4000  ...   0.3020  0.3176  0.3020

( 0 , 2 ,.,.) = 
  0.5294  0.5294  0.5294  ...   0.1451  0.1412  0.1333
  0.5255  0.5333  0.5373  ...   0.1725  0.1451  0.1412
  0.5373  0.5490  0.5451  ...   0.2314  0.1843

In [3]:
# Load annotations file for the training images.
mscoco_train = json.load(open('data/annotations/train_captions.json'))
train_ids = [entry['id'] for entry in mscoco_train['images']]
train_id_to_file = {entry['id']: 'data/train2014/' + entry['file_name'] for entry in mscoco_train['images']}

# Extract out the captions for the training images
train_id_set = set(train_ids)
train_id_to_captions = defaultdict(list)
for entry in mscoco_train['annotations']:
    if entry['image_id'] in train_id_set:
        train_id_to_captions[entry['image_id']].append(entry['caption'])

# Load annotations file for the validation images.
mscoco_val = json.load(open('data/annotations/val_captions.json'))
val_ids = [entry['id'] for entry in mscoco_val['images']]
val_id_to_file = {entry['id']: 'data/val2014/' + entry['file_name'] for entry in mscoco_val['images']}

# Extract out the captions for the validation images
val_id_set = set(val_ids)
val_id_to_captions = defaultdict(list)
for entry in mscoco_val['annotations']:
    if entry['image_id'] in val_id_set:
        val_id_to_captions[entry['image_id']].append(entry['caption'])

# Load annotations file for the testing images
mscoco_test = json.load(open('data/annotations/test_captions.json'))
test_ids = [entry['id'] for entry in mscoco_test['images']]
test_id_to_file = {entry['id']: 'data/val2014/' + entry['file_name'] for entry in mscoco_test['images']}

# Preprocessing

We do the same preprocessing done in assignment 3. 

In [4]:
sentences = [sentence for caption_set in train_id_to_captions.values() for sentence in caption_set]

# Lower-case the sentence, tokenize them and add <SOS> and <EOS> tokens
sentences = [["<SOS>"] + word_tokenize(sentence.lower()) + ["<EOS>"] for sentence in sentences]

# Create the vocabulary. Note that we add an <UNK> token to represent words not in our vocabulary.
vocabularySize = 1000
word_counts = Counter([word for sentence in sentences for word in sentence])
vocabulary = ["<UNK>"] + [e[0] for e in word_counts.most_common(vocabularySize-1)]
word2index = {word:index for index,word in enumerate(vocabulary)}
one_hot_embeddings = np.eye(vocabularySize)

# Build the word2vec embeddings
wordEncodingSize = 300
filtered_sentences = [[word for word in sentence if word in word2index] for sentence in sentences]
w2v = Word2Vec(filtered_sentences, min_count=0, size=wordEncodingSize)
w2v_embeddings = np.concatenate((np.zeros((1, wordEncodingSize)), w2v.wv.syn0))

# Define the max sequence length to be the longest sentence in the training data. 
maxSequenceLength = max([len(sentence) for sentence in sentences])

def preprocess_numberize(sentence):
    """
    Given a sentence, in the form of a string, this function will preprocess it
    into list of numbers (denoting the index into the vocabulary).
    """
    tokenized = word_tokenize(sentence.lower())
        
    # Add the <SOS>/<EOS> tokens and numberize (all unknown words are represented as <UNK>).
    tokenized = ["<SOS>"] + tokenized + ["<EOS>"]
    numberized = [word2index.get(word, 0) for word in tokenized]
    
    return numberized

def preprocess_one_hot(sentence):
    """
    Given a sentence, in the form of a string, this function will preprocess it
    into a numpy array of one-hot vectors.
    """
    numberized = preprocess_numberize(sentence)
    
    # Represent each word as it's one-hot embedding
    one_hot_embedded = one_hot_embeddings[numberized]
    
    return one_hot_embedded

def preprocess_word2vec(sentence):
    """
    Given a sentence, in the form of a string, this function will preprocess it
    into a numpy array of word2vec embeddings.
    """
    numberized = preprocess_numberize(sentence)
    
    # Represent each word as it's one-hot embedding
    w2v_embedded = w2v_embeddings[numberized]
    
    return w2v_embedded

def compute_bleu(reference_sentences, predicted_sentence):
    """
    Given a list of reference sentences, and a predicted sentence, compute the BLEU similary between them.
    """
    reference_tokenized = [word_tokenize(ref_sent.lower()) for ref_sent in reference_sentences]
    predicted_tokenized = word_tokenize(predicted_sentence.lower())
    return sentence_bleu(reference_tokenized, predicted_tokenized)

# 1. Setup Image Encoder

We load in the pre-trained VGG-16 model, and remove the final layer, as done in assignment 2.

In [7]:
# Your code goes here
vgg_model = models.vgg16(pretrained=True).cuda()

class VggMinusOneModel(torch.nn.Module):
    def __init__(self, vgg_model):
        """
        When constructing the model, we initialize two linear modules and assign them
        as class fields. We also, as done earlier, remove the final layer of the vgg model.
        """
        super(VggMinusOneModel, self).__init__()
        self.features = vgg_model.features
        self.classifier = nn.Sequential(*list(vgg_model.classifier.children())[:-1])
    
    def forward(self, x):
        """
        Pass the input through the network, applying the sigmoid activation function after each layer.
        """
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

vggMinusOne = VggMinusOneModel(vgg_model)
vggMinusOne


VggMinusOneModel(
  (features): Sequential(
    (0): Conv2d (3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d (64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (5): Conv2d (64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d (128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (10): Conv2d (128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d (256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d (256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (17): Conv2d (256, 512, kernel_

# 2. Setup a Language Decoder

We're going to reuse our decoder from Assignment 3.

In [37]:
# Your code goes here
use_cuda = True
class DecoderLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(DecoderLSTM, self).__init__()
        self.hidden_size = hidden_size

        self.lstm = nn.LSTM(input_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)

    def forward(self, input, hidden):
        output = F.relu(input)
        output, hidden = self.lstm(output, hidden)
        output = self.out(output)
        output = F.log_softmax(output.squeeze())
        return output.unsqueeze(0), hidden

    def initHidden(self, init_size, image_features):
        self.project = nn.Linear(init_size, self.hidden_size).cuda()
        result = self.project(image_features)
        # result = Variable(torch.zeros(1, 1, self.hidden_size))
        if use_cuda:
            return result.cuda()
        else:
            return result

# 3. Train encoder-decoder



In [92]:
use_cuda = True

# The next two functions are part of some other deep learning frameworks, but PyTorch
# has not yet implemented them. We can find some commonly-used open source worked arounds
# after searching around a bit: https://gist.github.com/jihunchoi/f1434a77df9db1bb337417854b398df1.
def _sequence_mask(sequence_length, max_len=None):
    if max_len is None:
        max_len = sequence_length.data.max()
    batch_size = sequence_length.size(0)
    seq_range = torch.arange(0, max_len).long()
    seq_range_expand = seq_range.unsqueeze(0).expand(batch_size, max_len)
    seq_range_expand = Variable(seq_range_expand)
    if sequence_length.is_cuda:
        seq_range_expand = seq_range_expand.cuda()
    seq_length_expand = (sequence_length.unsqueeze(1)
                         .expand_as(seq_range_expand))
    return seq_range_expand < seq_length_expand


def compute_loss(logits, target, length):
    """
    Args:
        logits: A Variable containing a FloatTensor of size
            (batch, max_len, num_classes) which contains the
            unnormalized probability for each class.
        target: A Variable containing a LongTensor of size
            (batch, max_len) which contains the index of the true
            class for each corresponding step.
        length: A Variable containing a LongTensor of size (batch,)
            which contains the length of each data in a batch.

    Returns:
        loss: An average loss value masked by the length.
    """
    # logits_flat: (batch * max_len, num_classes)
    logits_flat = logits.view(-1, logits.size(-1))
    # log_probs_flat: (batch * max_len, num_classes)
    log_probs_flat = F.log_softmax(logits_flat)
    # target_flat: (batch * max_len, 1)
    target_flat = target.view(-1, 1)
    # losses_flat: (batch * max_len, 1)
    losses_flat = -torch.gather(log_probs_flat, dim=1, index=target_flat)
    # losses: (batch, max_len)
    losses = losses_flat.view(*target.size())
    # mask: (batch, max_len)
    mask = _sequence_mask(sequence_length=length, max_len=target.size(1))
    losses = losses * mask.float()
    loss = losses.sum() / length.float().sum()
    return loss

def train(input_image,
          input_variables, 
          target_variables, 
          input_lens,
          encoder, 
          decoder, 
          encoder_optimizer, 
          decoder_optimizer, 
          criterion, 
          embeddings=one_hot_embeddings, 
          teacher_force=True,
          train_encoder=False):
    if train_encoder:
        encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_variables.size()[0]
    target_length = target_variables.size()[0]

    # Pass through the encoder
    image_features = encoder(input_image)
    
    
    # Construct the decoder input (initially <SOS> for every batch)
    decoder_input = Variable(torch.FloatTensor([[embeddings[word2index["<SOS>"]]
                                                for i in range(input_variables.size(1))]]))
    decoder_input = decoder_input.cuda() if use_cuda else decoder_input

    # Set the initial hidden state of the decoder to be the last hidden state of the encoder
    last_hidden = torch.stack([decoder.initHidden(image_features.size(1), image_features).squeeze() 
                               for i,length in enumerate(input_lens)]).unsqueeze(0)
    decoder_hidden = (last_hidden, last_hidden)

    # Prepare the results tensor
    all_decoder_outputs = Variable(torch.zeros(*input_variables.size()))
    if use_cuda:
        all_decoder_outputs = all_decoder_outputs.cuda()
        
    all_decoder_outputs[0] = decoder_input
        
    # Iterate over the indices after the first.
    for t in range(1,target_length):
        decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
    
        if random.random() <= 0.3:
            decoder_input = input_variables[t].unsqueeze(0)
        else:
            topv, topi = decoder_output.data.topk(1)
                       
            #Prepare the inputs
            decoder_input = torch.stack([Variable(torch.FloatTensor(embeddings[ni])).cuda()
                                         for ni in topi.squeeze()]).unsqueeze(0)
        
        # Save the decoder output
        all_decoder_outputs[t] = decoder_output
        
    loss = compute_loss(all_decoder_outputs.transpose(0,1).contiguous(),
                        target_variables.transpose(0,1).contiguous(), 
                        Variable(torch.LongTensor(input_lens)).cuda())

    loss.backward()
    
    torch.nn.utils.clip_grad_norm(encoder.parameters(), 10.0)
    torch.nn.utils.clip_grad_norm(decoder.parameters(), 10.0)

    if train_encoder:
        encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.data[0]

def pad_seq(arr, length, pad_token):
    """
    Pad an array to a length with a token.
    """
    if len(arr) == length:
        return np.array(arr)
    
    return np.concatenate((arr, [pad_token]*(length - len(arr))))

In [38]:
encoder = VggMinusOneModel(vgg_model)
decoder = DecoderLSTM(input_size=len(vocabulary), hidden_size=300, output_size=len(vocabulary)).cuda()

In [91]:
# Your code goes here
encoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=0.01) 
decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=0.01) 
criterion = nn.CrossEntropyLoss()  

num_epochs = 5
for _ in range(num_epochs):
    for i, train_id in enumerate(train_ids):
        # Get the sentences in the batch
        img = load_image(train_id_to_file[train_id])
        sentences = train_id_to_captions[train_id]
        
        # Get the sentence lengths
        sentence_lens = [len(preprocess_numberize(sentence)) for sentence in sentences]
        
        # Sort by the sentence lengths
        sorted_indices = sorted(list(range(len(sentence_lens))), key=lambda i: sentence_lens[i], reverse=True)
        sentences = [sentences[i] for i in sorted_indices if sentence_lens[i] > 0]
        
        # Filter out 0 sentence lengths
        sentence_lens = [sentence_lens[i] for i in sorted_indices if sentence_lens[i] > 0]
        
        # Determine length to pad everything to
        max_len = max(sentence_lens)
        
        # Preprocess all of the sentences in each batch
        one_hot_embedded_list = [preprocess_one_hot(sentence) for sentence in sentences]
        one_hot_embedded_list_padded = [pad_seq(embed, max_len, np.zeros(len(vocabulary))) 
                                        for embed in one_hot_embedded_list]
                
        numberized_list = [preprocess_numberize(sentence) for sentence in sentences]
        numberized_list_padded = [pad_seq(numb, max_len, 0).astype(torch.LongTensor) for numb in numberized_list]
                
        # Convert to variables
        input_variable = Variable(torch.FloatTensor(one_hot_embedded_list_padded)).cuda()
        target_variable = Variable(torch.LongTensor(numberized_list_padded)).cuda()
        
        # Transpose from batch_size x max_seq_len x vocab_size to max_seq_len x batch_size x vocab_size
        input_variable = input_variable.transpose(0, 1)
        target_variable = target_variable.transpose(0, 1)

        loss = train(img,
                     input_variable,
                     target_variable, 
                     sentence_lens,
                     encoder,
                     decoder, 
                     encoder_optimizer,
                     decoder_optimizer, 
                     criterion)
        
        if i % 100 == 0:
            print(i,loss)


Variable containing:
( 0 ,.,.) = 
  0.9257  1.1044 -0.9747  ...   0.6481 -0.3852  0.9975
 -0.6596 -0.7308  0.5060  ...   1.1433  0.7249  0.2053
 -1.3106  0.7272 -0.8236  ...   0.4914 -0.1970  0.1099
  0.9525 -0.4777 -0.0022  ...   1.3745 -0.0420  0.1207
 -1.1089 -0.4431  1.0188  ...   0.6058  0.0268 -0.4125
[torch.cuda.FloatTensor of size 1x5x300 (GPU 0)]



C:\Users\belin\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  from ipykernel import kernelapp as app
C:\Users\belin\Anaconda3\lib\site-packages\ipykernel_launcher.py:38: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


0 4.510356903076172
Variable containing:
( 0 ,.,.) = 
  0.6390 -0.1767 -0.1796  ...  -0.0269 -0.3592 -0.7425
 -0.9217 -0.2325 -0.6827  ...  -0.1601  0.7287 -0.2234
  0.4745 -1.2511 -1.5534  ...   0.3004  0.3927  0.1781
  0.3072  0.6949 -0.3287  ...   0.2721 -0.0474  0.2230
 -0.3866 -0.2335 -0.6049  ...  -0.3465 -0.3086  0.6670
[torch.cuda.FloatTensor of size 1x5x300 (GPU 0)]

Variable containing:
( 0 ,.,.) = 
  0.3433  0.5123 -0.2208  ...  -0.0675  0.3179  0.4164
  0.4636 -1.3132  1.2998  ...  -0.0678 -0.3261 -0.4601
 -0.5256  1.1358 -1.1867  ...   0.3347 -0.1623 -0.2360
 -0.0414  0.0781  0.2953  ...   0.8919  0.2110 -0.6048
  0.2293 -0.6745  0.1576  ...   1.1106  0.0052  0.2283
[torch.cuda.FloatTensor of size 1x5x300 (GPU 0)]

Variable containing:
( 0 ,.,.) = 
 -0.7908  0.2930  0.1519  ...  -0.3262 -0.1838  0.4456
  0.0562 -0.0975  0.2607  ...  -0.2443  0.1781 -0.0278
  0.3330 -0.1083  0.3781  ...   0.0572  0.6487  0.1218
  0.7294  0.6381  0.4648  ...   0.0119  0.4912  0.1718
  0.3615

KeyboardInterrupt: 

In [89]:
def seq2seq_inference(input_image, embeddings=one_hot_embeddings, max_length=20):
    image_features = encoder(input_image)

    # Construct the decoder input (initially <SOS> for every batch)
    decoder_input = Variable(torch.FloatTensor([[embeddings[word2index["<SOS>"]]]]))
    decoder_input = decoder_input.cuda() if use_cuda else decoder_input

    # Set the initial hidden state of the decoder to be the last hidden state of the encoder
    last_hidden = decoder.initHidden(image_features.size(1), image_features).squeeze() 
    print(last_hidden)
    decoder_hidden = (last_hidden, last_hidden)
    
    # Iterate over the indices after the first.
    decoder_outputs = []
    for t in range(1,max_length):
        decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
    
        # Get the top result
        topv, topi = decoder_output.data.topk(1)
        ni = topi[0][0]
        decoder_outputs.append(ni)

        if vocabulary[ni] == "<EOS>":
            break
        
        #Prepare the inputs
        decoder_input = Variable(torch.FloatTensor([[embeddings[ni]]])).cuda()
        decoder_input = decoder_input.cuda() if use_cuda else decoder_input

    return ' '.join(vocabulary[i] for i in decoder_outputs)

print(seq2seq_inference(load_image(train_id_to_file[train_ids[0]])))
print(seq2seq_inference(load_image(train_id_to_file[train_ids[1]])))
print(seq2seq_inference(load_image(train_id_to_file[train_ids[2]])))
print(seq2seq_inference(load_image(train_id_to_file[train_ids[3]])))
print(seq2seq_inference(load_image(train_id_to_file[train_ids[4]])))



Variable containing:
 0.6900
 0.0247
-0.0205
-1.0356
-0.3491
-0.1649
 0.2792
 0.1113
-0.3156
-0.2398
 0.5470
 0.4465
-0.6144
-0.0315
-0.3382
 0.1465
-1.1278
 0.2357
 1.1728
-0.1929
 0.8628
 0.0968
-0.7863
 0.8773
-0.5609
-0.3402
 0.1330
-0.3282
-0.3592
-0.0334
 0.3048
 0.2350
-0.2895
-0.1549
 2.1194
 0.5573
-0.3420
-1.3992
-0.3550
 0.8151
 0.5322
-0.0855
 0.1680
-0.8578
-1.1371
-0.3064
-1.4733
 1.3607
-0.4748
-0.6319
 1.1919
 0.2849
-0.3120
 1.0487
-0.4046
 0.7699
 0.3196
 0.2301
-0.0129
-1.1962
 0.1642
-0.1505
 0.3206
 1.0369
 0.0443
-0.1003
 1.0937
-0.1749
 0.1556
 0.5714
 0.2870
 1.0912
-0.3538
 0.7328
 0.2391
-1.5871
-0.2994
-0.9087
-0.1741
-0.2047
-0.4771
-1.1908
-0.0373
 1.4655
-0.2772
 1.1481
-1.1461
 1.2535
 0.8086
-2.2125
-0.0352
-0.0641
 0.1014
-0.8950
 0.9669
-0.4035
-1.6101
 0.8782
 0.3269
 0.4748
 0.1439
-0.3359
 0.8900
-0.7650
-0.6208
-0.1399
-0.2916
 1.0608
-0.0043
-0.2952
 0.5623
 0.1293
-0.9467
-1.7711
-0.7598
-0.0122
-0.2835
 1.0413
 0.6737
-0.5844
-1.1578
-0.0541
 0.

RuntimeError: Expected hidden size (1, 1, 300), got (300,)

# 4. MAP and Sampling Inference


In [ ]:
# Your code goes here

# 5. Evaluate performance

For validation images compute the average BLEU score.

In [ ]:
# Your code goes here